In [17]:
exec(open("cleanup.py").read())

In [2]:
import json
from collections import defaultdict

def get_response(fname):
    with open(fname, 'r') as f:
        data = f.read()

    response = json.loads(data)

    surv_resp = defaultdict(dict)
    pass_resp = defaultdict(str)
    time_resp = defaultdict(str)

    for k, v in response.items():
        if 'response' in k:
            temp = k.split("_")
            pid = temp[0].strip('p')
            v = json.loads(v)
            surv_resp[pid].update(v)
        elif 'time' in k:
            temp = k.split("_")
            pid = temp[0]
            time_resp[pid] = float(v)
        elif 'password' in k:
            pass_resp[k] = v

    return (pass_resp, time_resp, surv_resp)

In [3]:
page_map = {
    '0': 'startstudy',
    '1': 'fpass',
    '2': 'fcog',
    '3': 'fpasssurvey',
    '4': 'fpassrecall',
    '5': 'pagegif',
    '6': 'desinint',
    '7': 'quest',
    '8': 'csquest',
    '9': 'motivation',
    '10': 'spass',
    '11': 'scog',
    '12': 'spasssurvey',
    '13': 'spassrecall',
    '14': 'demo',
    '15': 'thanks',
}

In [ ]:
cat_map = {
    'Perspicuity': [1, 2, 3, 4],
    'Stimulation': [5, 7, 8, 9],
    'Novelty': [10, 11, 12, 13],
    'Aesthetics': [14, 16, 17, 18],
    'Usefulness': [19, 20, 21, 22],
    'Clarity': [23, 24, 25, 26],
    'Engagement': [27],
    'Informative': [28],
    'Attention': [6, 15, 29]
}

In [32]:
import os
fpath = './responses'
password_response = []
time_response = []
survey_response = []
for i, fname in enumerate(os.listdir(fpath)):
    pss, tim, sur = get_response(os.path.join(fpath, fname))
    password_response.append(pss)
    time_response.append(tim)
    survey_response.append(sur)

In [33]:
import pandas as pd
password_response_df = pd.DataFrame(password_response)
password_response_df.sort_index(axis=1)

,password1,password1_recall1,password2,password2_recall1,password2_recall2,password2_recall3
0,ramropassword@09,NaN,malekosaasan@1994,malekosasan@1994,maalekosaasan@19944,maalekosaasan@1994
1,bankkoaccount@123,bankkoaccount@1233,ishouldcreatestrongpassword@123,NaN,NaN,NaN


In [42]:
page_time_df = pd.DataFrame(time_response)
page_time_df.sort_index(axis=1, key=lambda x: x.astype(int))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,15.353,35.620,4.721,76.599,13.446,16.145,68.430,148.547,129.065,12.815,78.915,8.332,59.734,29.451
1,26.206,37.073,26.425,78.942,26.393,14.027,32.727,175.211,93.015,22.265,44.518,19.347,92.398,20.939


In [43]:
# top 5 pages with higher time
top5 = pd.DataFrame(columns=['page', 'time'])
for i, data in page_time_df.mean().sort_values(ascending=False).head(5).items():
    top5 = top5.append(pd.DataFrame([[page_map.get(i), data]], columns=top5.columns), ignore_index=True)

top5

,page,time
0,quest,161.8790
1,csquest,111.0400
2,fpasssurvey,77.7705
3,spasssurvey,76.0660
4,spass,61.7165


In [16]:
pd.DataFrame(survey_response)

,7,12,3,8,14
0,"{'1-21': '6', '1-25': '5', '1-23': '4', '1-24'...","{'1-1': '7', '2-1': '6', '2-2': '1', '2-3': '1...","{'1-1': '7', '2-1': '1', '2-2': '2', '2-3': '1...","{'4-1': '3', '6-1': ['Email', 'Financial Websi...","{'1-1': 'Woman', '2-1': '25-29 years old', '3-..."
